In [1]:
import ipywidgets as widgets
import horizon_api
import json

MAPA_TELES = "telesa/"

# Simulacija nebesnih teles

## Prenos nebesnih teles
Izberite datum, za katerega želite prenesti podatke o nebesnih telesih in s pritiskom na gumb generirajte datoteko za izbrano nebesno telo.

In [10]:
slovar_teles = horizon_api.search_body()
datum = input("zapišite izbran datum v formatu yyyy-mm-dd")

text_ime_telesa = widgets.Text(description="Ime telesa")
dropdown_izbira = widgets.Dropdown(
    options=[],
    description='Izbira telesa:',
)
out = widgets.Output(layout={'border': '1px solid black', 'padding': "10px"})

# populira dropdown z najdenimi ključi za iskan niz
def najdi_telo(t):
    ime_telesa = str(t.value).capitalize()
    
    if ime_telesa in slovar_teles.keys():   najdeno = {ime_telesa: slovar_teles[ime_telesa]}
    else:                                   najdeno = horizon_api.search_body(ime_telesa)
    
    out.clear_output(); dropdown_izbira.value=None
    if not najdeno: 
        with out: print("Žal nismo našli nobeno telo z tem ključem...") 
    else:
        with out: print("Izberite želeno telo s seznama") 
        dropdown_izbira.options = najdeno.keys()

# na disku ustvari datoteko ki predstavlja izbrano telo
def ustvari_telo(t):
    if t['new'] != t['old'] and t['new']:
        name = t['new']
        json_data = horizon_api.get_body_json(slovar_teles[name], datum, name=name)

        with open(f"{MAPA_TELES}telo_{name}.json", "w", encoding="utf-8") as f:
            json.dump(json_data, f, indent=4)
        out.clear_output()

        if json_data['mass'] == 0 or json_data["radius"] == 0 or len(json_data["x_vec"]) == 0 or len(json_data["v_vec"]) == 0:
            with out: print("Telo ustvarjeno! (manjkajo ključni podatki)")
        else:
            with out: print("Telo ustvarjeno!") 
        

text_ime_telesa.on_submit(najdi_telo)
dropdown_izbira.observe(ustvari_telo, names="value")

display(text_ime_telesa)
display(dropdown_izbira)
display(out)

Text(value='', description='Ime telesa')

Dropdown(description='Izbira telesa:', options=(), value=None)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

(649586681.4289607, 355992334.4863436, -15987306.73162231)
